In [4]:
str_data = ['RDD data 1', 'RDD data 2']
#rdd_data = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rdd_data = sc.parallelize(str_data)

data_from_collect = rdd_data.collect()
data_from_first = rdd_data.first()

print(data_from_collect)
print(data_from_first)

# 읽어온 데이터를 다른 경로에 저장
data_in_hdfs = sc.textFile("/sorting_input/sorting.txt")
print(data_in_hdfs.collect())

# 저장된 데이터 살펴보기
data_in_hdfs.saveAsTextFile("/tmp/insoo67_park/sorting_input")
data_in_hdfs_read = sc.textFile("/tmp/insoo67_park/sorting_input/*")
print(data_in_hdfs_read.collect())

['RDD data 1', 'RDD data 2']
RDD data 1
['82, 58, 49, 99, 25, 37, 44, 94, 70, 18, 73, 81, 32, 21, 79, 13, 38, 51, 70, 38, 82, 43, 42', '63, 78, 39, 1, 56, 35, 93, 66, 5, 91, 77, 86, 45, 58, 77, 49, 47, 15, 92, 12', '70, 38, 49, 58, 96, 79, 69, 94, 50, 11, 86, 39, 58, 74, 69, 82, 97', '84, 92, 92, 80, 98, 44, 27, 55, 52, 47, 44, 96, 64, 37, 30, 18, 18, 49, 53, 43', '84, 68, 38, 78, 23, 16, 41, 55, 50, 89, 9, 78, 23, 95, 24, 29, 21, 4, 71']
['82, 58, 49, 99, 25, 37, 44, 94, 70, 18, 73, 81, 32, 21, 79, 13, 38, 51, 70, 38, 82, 43, 42', '63, 78, 39, 1, 56, 35, 93, 66, 5, 91, 77, 86, 45, 58, 77, 49, 47, 15, 92, 12', '70, 38, 49, 58, 96, 79, 69, 94, 50, 11, 86, 39, 58, 74, 69, 82, 97', '84, 92, 92, 80, 98, 44, 27, 55, 52, 47, 44, 96, 64, 37, 30, 18, 18, 49, 53, 43', '84, 68, 38, 78, 23, 16, 41, 55, 50, 89, 9, 78, 23, 95, 24, 29, 21, 4, 71']


In [9]:
no_files_RDD = sc.textFile("no_files") # RDD 연결만 만듦
print(no_files_RDD) # error 발생 안 함

#no_files_RDD.collect() # 연산 수행, error 발생
#no_files_RDD.count()

no_files MapPartitionsRDD[30] at textFile at NativeMethodAccessorImpl.java:0


In [47]:
#Exercise 1-1
A = sc.parallelize(['M_store', 'S_cafe', 'M_store', 'S_cafe', 'H_shop', 'D_store'])
B = sc.parallelize(['S_cafe', 'M_store', 'S_cafe', 'A_market', 'A_market'])

A_B_iter = A.intersection(B).collect()

def f(x):
    return x in A_B_iter

A_visit = A.filter(f)
B_visit = B.filter(f)

A_visit_countByValue = A_visit.countByValue()
B_visit_countByValue = B_visit.countByValue()

print(A_visit_countByValue)
print(B_visit_countByValue)

defaultdict(<class 'int'>, {'M_store': 2, 'S_cafe': 2})
defaultdict(<class 'int'>, {'S_cafe': 2, 'M_store': 1})


In [49]:
#Exercise 1-2
all_data = sc.parallelize(['success', 'success', 'error:no_signal', 'success', 'error:no_material'])

error_list = all_data.filter(lambda x : x[0] == "e")
error_reason = error_list.map(lambda s: s.split(":")[1])

print(error_reason.collect())

['no_signal', 'no_material']


In [55]:
#Exercise 1-3
all_data = sc.parallelize(["Samsung:72", "Huawei:59", "Apple:42"])
shipment = all_data.map(lambda s: int(s.split(":")[1]))
sum_shipment = shipment.reduce(lambda a, b: a+b)
print(sum_shipment, "Mil.")

173 Mil.


In [58]:
#Exercise 1-4
all_data = sc.parallelize(["Galaxy,A10,30.3", "Galaxy,A50,24.2", "Galaxy,A20,19.2", "iPhone,XR,46.3", "iPhone,11,37.3", "Redmi,Note7,16.4"])

Galaxy_data = all_data.filter(lambda x: x.split(",")[0] == "Galaxy")
Galaxy_shipment = Galaxy_data.map(lambda s: float(s.split(",")[2]))

Galaxy_shipment_mean = Galaxy_shipment.mean()
print(Galaxy_shipment_mean, 'Mil.')

24.566666666666666 Mil.


In [66]:
# Exercise 2-1
from operator import add
RDD = sc.parallelize([("Note20", 30), ("Note20", -4), ("s20", -12), ("s20", 20), ("s20", -2), ("s20", -3), ("s20", 32), ("a21s", -19), ("a21s", 67)])

#print(RDD.groupByKey().mapValues(list).collect())
result = RDD.reduceByKey(add).sortByKey().collect()
print(result)

[('Note20', 26), ('a21s', 48), ('s20', 35)]


In [69]:
# Exercise 2-2
RDD = sc.parallelize(["CS, Seoul", "CS, Suwon", "CS, Busan", "CS, Daejeon", "R&D, seoul", "R&D, Daejeon"])

def f(x):
    xx = x.split(",")
    return (xx[0], xx[1])

group_by_key_list = RDD.map(f).groupByKey().mapValues(list).collect()
group_by_key_count = RDD.map(f).groupByKey().mapValues(len).collect()

print(group_by_key_list)
print(group_by_key_count)

[('CS', [' Seoul', ' Suwon', ' Busan', ' Daejeon']), ('R&D', [' seoul', ' Daejeon'])]
[('CS', 4), ('R&D', 2)]


In [78]:
# Exercise 2-3
A = sc.parallelize([('Thriller', 30), ('Everybody', 34), ('Everybody', 30), ('Billie_Jean', 12), ('Billie_Jean', 2)])
B = sc.parallelize([('Thriller', 20), ('Sorry', 23), ('Sorry', 3), ('Billie_Jean', 5)])

# A_groupByKey = A.groupByKey().mapValues(list)
# B_groupByKey = B.groupByKey().mapValues(list))

# A_B_iter = A_groupByKey.intersection(B_groupByKey).collect()
# print(A_B_iter)

# def f(x):
#    return x in A_B_iter

# A_saw = A_groupByKey.filter(f)
# B_saw = B_groupByKey.filter(f)

# A_sawt_countByValue = A_saw.countByValue()
# B_saw_countByValue = B_saw.countByValue()

# print(A_sawt_countByValue)
# print(B_saw_countByValue)
from operator import add
A_reduceByKey = A.reduceByKey(add)
B_reduceByKey = B.reduceByKey(add)

AB_join = A_reduceByKey.join(B_reduceByKey)

def f(x):
    return sum(x)

AB_join_mapValues = AB_join.mapValues(f).collect()
print(AB_join_mapValues)

[('Thriller', 50), ('Billie_Jean', 19)]


In [89]:
# Exercise 3-1
from pyspark.sql import Row
DF1 = sc.parallelize(
[Row(phase='packing', model='flex', serial='flex1'),
Row(phase='packing',  model='flex', serial='flex2'),
Row(phase='packing',  model='ion', serial='ion1'),
Row(phase='packing',  model='flex', serial='flex3'),
Row(phase='packing',  model='ion', serial='ion2'),
Row(phase='inspection',  model='flex', serial='flex4'),
Row(phase='inspection',  model='ion', serial='ion3'),
Row(phase='inspection',  model='flex', serial='flex5'),
Row(phase='inspection',  model='flex', serial='flex6'),
]).toDF()

DF2 = sc.parallelize(
[
    Row(phase='assembly', model='flex', serial='flex7'),
    Row(phase='assembly',  model='ion', serial='ion4')
]
).toDF()
DF1.unionByName(DF2).select("model", "phase").groupBy("model").count().show()

+-----+-----+
|model|count|
+-----+-----+
| flex|    7|
|  ion|    4|
+-----+-----+



In [90]:
# Homework
A_store = sc.parallelize([("Note20", 30), ("Note20", -4), ("s21", -12), ("s21", 20), ("s21", -2), ("s21", -3), ("s21", 32), ("a21s", -19), ("a21s", 67)])
B_store = sc.parallelize([("Note20", 20), ("Note20", -3), ("s21", -10), ("s21", 20), ("s21", -1), ("s21", -6), ("s21", 10), ("a21s", -9), ("a21s", 60)])

from operator import add
A_reduceByKey = A_store.reduceByKey(add)
B_reduceByKey = B_store.reduceByKey(add)

AB_join = A_reduceByKey.join(B_reduceByKey)

def f(x):
    return sum(x)

AB_join_mapValues = AB_join.mapValues(f).collect()
print(AB_join_mapValues)

[('Note20', 43), ('s21', 48), ('a21s', 99)]
